## Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Creating CoT

In [ ]:
import time
import traceback
import json
import os
from google import genai
from google.genai import types
from google.colab import userdata, drive

drive.mount('/content/drive')

output_dir = "/content/drive/MyDrive/ColabResults"
os.makedirs(output_dir, exist_ok=True)  

api_keys = [
    userdata.get('GEMINI_API_KEY_1'),
    userdata.get('GEMINI_API_KEY_2'),
    userdata.get('GEMINI_API_KEY_3')
]

def load_input(input_file):
     with open(input_file, "r", encoding="utf-8") as f:
        return json.load(f)

def get_system_inst():
    return types.Part.from_text(text="""
       You are a legal expert specializing in the Swedish legal system, with deep expertise in contract law (Avtalslagen). Your task is to generate a detailed chain-of-thought (CoT) annotation that clearly and thoroughly explains the reasoning process behind the provided answer to a legal question. Your response should follow the structured outline below:

1. **Understanding the Core Question:**
   - Briefly restate the question in your own words.
   - Identify the main legal issue(s) involved (e.g., the binding nature of a contract under duress).

2. **Identification of Relevant Legal Area:**
   - Specify the legal area (e.g., contract law) and relevant legislation (e.g., Lag (1915:218) om avtal och andra rättshandlingar på förmögenhetsrättens område).
   - Mention the key terms (such as "avtal," "bindande," "tvingad," "tvång," "ogiltigt") and their significance.

3. **Scanning and Locating Relevant Provisions:**
   - Describe the process of scanning the legislation for relevant keywords (e.g., “tvång”, “ogiltighet”) and identify the sections that deal with duress.
   - Clearly indicate the relevant sections (e.g., Section 29 and, if applicable, Section 28) and explain why these are critical.

4. **Detailed Analysis of the Identified Statutes:**
   - For Section 29:
     - Present the exact legal text or a close paraphrase.
     - Break down the key elements (e.g., “rättshandling tvungen,” “genom våld eller hot,” “icke gällande mot den tvungne”).
     - Explain the implications (e.g., that contracts entered under severe duress are void).
   - For Section 28 (if relevant for nuance):
     - Explain how it covers milder forms of duress or different scenarios (e.g., third-party duress or cases involving good faith).
     - Compare and contrast with Section 29, highlighting the differences in legal consequence (void versus voidable).

5. **Logical Reasoning Steps:**
   - Provide a step-by-step explanation that connects the legal principles to the final answer:
     - **Step 1:** Identify the legal issue based on the question.
     - **Step 2:** Determine the applicable legal framework (mentioning Avtalslagen).
     - **Step 3:** Locate the specific provisions (Section 29, and Section 28 if needed) relevant to duress.
     - **Step 4:** Analyze the text of these provisions, breaking down the elements.
     - **Step 5:** Apply the analysis to the scenario presented in the question.
     - **Step 6:** Conclude with the reasoning that leads to the final answer.

6. **Consideration of Exceptions or Nuances:**
   - Discuss any potential exceptions (e.g., differences between severe duress and other forms of duress).
   - Mention the role of factors such as the counterparty’s knowledge or good faith.
   - Briefly address the burden of proof and any relevant consequences (such as restitution).

7. **Synthesis and Structured Conclusion:**
   - Summarize the entire reasoning process in a clear, concise conclusion.
   - Reaffirm how the reasoning supports the final answer (e.g., that an agreement entered under duress is invalid as per Section 29).

8. **Formatting and Clarity:**
   - Use clear headings, bullet points, or numbered steps where appropriate.
   - Maintain a formal, expert tone and precise legal terminology throughout the response.

Your final output should be comprehensive and self-contained, so that any reader with a legal background can follow the entire reasoning process without additional context. Do not add any extraneous commentary—focus solely on the logical, step-by-step reasoning process.

**Input Placeholders:**

**Question:**
[QUESTION]

**Answer:**
[ANSWER]

---

Now, using the structure above, generate the detailed chain-of-thought reasoning annotation for the provided question and answer.
Your entire response must be written in correct, legal Swedish. Use a formal tone appropriate for Swedish legal professionals and legal academics. Do not use English at any point.
    """)

def save_output(results, output_file):
    with open(output_file, "w", encoding="utf-8") as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

def generate_cot(qa_pairs, output_file):
    model = "gemini-2.0-flash-thinking-exp-01-21"  
    results = []

    batch_size = 9
    tot_batches = (len(qa_pairs) + batch_size - 1) // batch_size  

    for batch_num in range(tot_batches):
        current_key = api_keys[batch_num % len(api_keys)]
        client = genai.Client(api_key=current_key)

        batch = qa_pairs[batch_num * batch_size : (batch_num + 1) * batch_size]

        system_instruction = get_system_inst()

        for item in batch:
            question = item["question"]
            answer = item["answer"]

            input_text = f"""
            **Fråga:**
            {question}

            **Svar:**
            {answer}
            """
            contents = [
                types.Content(role="user", parts=[types.Part.from_text(text=input_text)])
            ]
            content_config = types.GenerateContentConfig(
                response_mime_type="text/plain",
                system_instruction=[system_instruction]
            )

            print(f"\nBatch...")
            try:
                response = client.models.generate_content(
                    model=model,
                    contents=contents,
                    config=content_config,
                )

                chain_of_thought = response.text.strip()
                print(chain_of_thought)

                results.append({
                    "question": question,
                    "answer": answer,
                    "chain_of_thought": chain_of_thought
                })
            except Exception:
                print(f"An error occured")
                traceback.print_exc()
                continue  

        if batch_num < tot_batches - 1:
            print("\nPause 60s...")
            time.sleep(60)

    save_output(results, output_file)

    print(f"\nResults saved to our output file!")

if __name__ == "__main__":
    input_file = "/content/QnA_noDuplicate.json"  
    output_file = os.path.join(output_dir, "output_results.json")  

    qa_data = load_input(input_file)

    print(f"Loaded qa's from input file.")

    generate_cot(qa_data, output_file)
